# Project: Advanced Lane Finding - Main

### <font color='red'> Note: Run 'Advanced_LaneFinding - PreProcessing' to generate camera calibration, perspective transform matrices before running this! </font>

### Pipeline procedure
1. Load camera calibration, perspective transform matrices
2. Gradient & color filtering
3. Undistort & unwarp
4. Find lane lines:
    a) Use sliding window for first image or when reset is needed
    b) Use lookback for subsequent images

### Import libraries

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import pickle

%matplotlib inline

In [2]:
def apply_gradx_color_filter_old(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    """Apply sobel filter along x and s-channel color filter"""
    img = np.copy(img)

    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]

    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1

    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    # # Stack each channel
    # color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    combined = np.zeros_like(s_channel)
    combined[((sxbinary == 1)  | (s_binary == 1))] = 1

    return combined

def apply_gradx_color_filter(img, s_thresh=(170, 255), sx_thresh=(100, 255), l_thresh=(170, 255)):
    """Apply sobel filter along x and s-channel color filter"""
    img = np.copy(img)
    
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    mask_yellow = cv2.inRange(img, (128, 128, 0), (255, 255, 90))
    l_channel[mask_yellow > 128] = 255
    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Threshold color channel
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_thresh[0]) & (l_channel <= l_thresh[1])] = 1
    
    # # Stack each channel
    # color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    
    combined = np.zeros_like(s_channel)
    combined[((sxbinary == 1)  | (s_binary == 1) | (l_binary == 1))] = 1
    
    return combined

def undistort_image(img, mtx, dist):
    return cv2.undistort(img, mtx, dist, None, mtx)

def warp_perspective(img, M):
    """
    `img` is the undistored image
    `M` is the perspective transform matrix
    """
    warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]), 
                                      flags=cv2.INTER_LINEAR)
    return warped

def find_lane_pixels(binary_warped, nwindows=9, margin=100, min_pixels=50):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height

        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this

        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
                              (win_xleft_high,win_y_high),
                      (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
                              (win_xright_high,win_y_high),
                      (0,255,0), 2)

        ### indIdentify the nonzero pixels in x and y within the window ###
        good_left_inds = np.where((nonzerox >= win_xleft_low) &
                                  (nonzerox < win_xleft_high) &
                                  (nonzeroy >= win_y_low) &
                                  (nonzeroy < win_y_high))[0]
        # list(zip(nonzerox[nz_left], nonzeroy[nz_left]))
        good_right_inds = np.where((nonzerox >= win_xright_low) &
                                  (nonzerox < win_xright_high) &
                                  (nonzeroy >= win_y_low) &
                                  (nonzeroy < win_y_high))[0]

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        ### If you found > min_pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > min_pixels:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))

        if len(good_right_inds) > min_pixels:
            rightx_current = int(np.mean(nonzerox[good_right_inds]))


    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

def search_around_poly(binary_warped, left_fit, right_fit, margin=100):
    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = np.where((nonzerox >= (np.polyval(left_fit, nonzeroy) - margin)) &
                              (nonzerox <= (np.polyval(left_fit, nonzeroy) + margin)))
    right_lane_inds = np.where((nonzerox >= (np.polyval(right_fit, nonzeroy) - margin)) &
                              (nonzerox <= (np.polyval(right_fit, nonzeroy) + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty

def fit_lane_polynomials(ylen, leftx, lefty, rightx, righty, order=2):
    ### Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty, leftx, order)
    right_fit = np.polyfit(righty, rightx, order)

    # Generate x and y values for plotting
    ploty = np.linspace(0, ylen-1, ylen)
    left_fitx = np.polyval(left_fit, ploty)
    right_fitx = np.polyval(right_fit, ploty)

    return left_fit, right_fit, left_fitx, right_fitx, ploty

def get_poly_lanes(img, prev_left_fit=None, prev_right_fit=None, margin=100):
    # Apply Sobel Gradient filter, color filter
    img_filt = apply_gradx_color_filter(img)

    # Undistort and unwarp image
    img_undistort = undistort_image(img=img_filt, mtx=mtx, dist=dist)
    img_unwarp = warp_perspective(img=img_undistort, M=M)

    # Find lane pixels
    if prev_left_fit is None or prev_right_fit is None:
        leftx, lefty, rightx, righty, out_img = find_lane_pixels(img_unwarp)
    else:
        # detect lines around previous fit
        leftx, lefty, rightx, righty = search_around_poly(img_unwarp, left_fit=prev_left_fit,
                                                          right_fit=prev_right_fit, margin=margin)

    # Fit polynomials lines to left, right lanes
    if len(leftx) and len(rightx):
        left_fit, right_fit, left_fitx, right_fitx, ploty = fit_lane_polynomials(img.shape[0], leftx, lefty, rightx, righty)
    else:
        left_fit = right_fit = left_fitx = right_fitx = ploty = None

    return img_unwarp, left_fit, right_fit, left_fitx, right_fitx, img_filt

def plot_lane_lines(img, left_x, right_x, y):
    for i in range(len(left_x)-1):
        cv2.line(img, (int(left_x[i]), int(y[i+1])), (int(left_x[i+1]), int(y[i+1])), color=(255, 0, 0), thickness=5)
        cv2.line(img, (int(right_x[i]), int(y[i+1])), (int(right_x[i+1]), int(y[i+1])), color=(255, 0, 0), thickness=5)

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.

    `initial_img` should be the image before any processing.

    The result image is computed as follows:

    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

def add_lanes_poly(image, left_fitx, right_fitx, y):
    # create lanes poly
    pts = np.vstack([np.vstack([left_fitx, y]).astype(np.int32).T, 
                     np.vstack([right_fitx[::-1], y[::-1]]).astype(np.int32).T])
    lanes_poly = np.zeros([image.shape[0], image.shape[1], 3])
    lanes_img = cv2.fillPoly(lanes_poly,  [pts],  (255, 0, 0))

    # unwarp lanes perspective
    lanes_img_warp = warp_perspective(img=lanes_img, M=np.linalg.inv(M)).astype(np.uint8)


    return weighted_img(img=lanes_img_warp, initial_img=image, α=1.0, β=0.5)

def calc_poly_curve_radius(poly_fit, y_eval):
    return ((1 + (2*poly_fit[0]*y_eval+poly_fit[1])**2)**1.5)/(2*abs(poly_fit[0]))

def calc_lane_curve_radius(poly_fit, y_eval, xm_per_pix, ym_per_pix):
    # convert poly fit from pixels to m
    real_fit = np.copy(poly_fit)
    real_fit[0] *= xm_per_pix/(ym_per_pix**2)
    real_fit[1] *= (xm_per_pix/ym_per_pix)
    y_eval_m = y_eval*ym_per_pix
    
    return calc_poly_curve_radius(real_fit, y_eval_m)

font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (50, 50)
bottomLeftCornerOfText2 = (50, 100)
bottomLeftCornerOfText3 = (50, 150)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2

class LaneFinding():
    def __init__(self, xm_per_pix=3.7/700., ym_per_pix=36/720., n_hist=12):
        self.ym_per_pix = ym_per_pix
        self.xm_per_pix = xm_per_pix
        self.n_hist = n_hist
        
        # was the line detected in the last iteration?
        self.detected = False  
        
        self.best_x_vals = [None, None]
        self.recent_x_vals = [[None, None]]
        
        self.best_fits = [None, None]  
        self.recent_fits = [[None, None]]
        
        #radius of curvature of the line in some units
        self.radius_of_curvatures = [[None, None]]
        self.best_radius = None
        
        #distance in meters of vehicle center from the line
        self.line_base_pos = [None]
        
        self.frame_count = 0
        
    def update_best_fits(self):
        if len(self.recent_fits) > self.n_hist:
            self.recent_fits = self.recent_fits[-self.n_hist:]
            self.recent_x_vals = self.recent_x_vals[-self.n_hist:]
            self.radius_of_curvatures = self.radius_of_curvatures[-self.n_hist:]
            self.line_base_pos = self.line_base_pos[-self.n_hist:]
        
        recent_fits_left = np.array([fits[0] for fits in self.recent_fits if fits[0] is not None])
        recent_fits_right = np.array([fits[1] for fits in self.recent_fits if fits[1] is not None])
        
        best_fit_left = np.average(recent_fits_left, 
                                   weights=range(len(recent_fits_left), 0, -1), 
                                   axis=0)
        best_fit_right = np.average(recent_fits_right,
                                    weights=range(len(recent_fits_right), 0, -1),
                                    axis=0)
        
        if not np.all(np.isnan(best_fit_left)) and not np.all(np.isnan(best_fit_right)):
            self.best_fits = [best_fit_left, best_fit_right]
#         else:
#             ELSE USE PREVIOUS FITS
#             self.best_fits = [None, None]

        recent_xvals_left = np.array([xvals[0] for xvals in self.recent_x_vals if xvals[0] is not None])
        recent_xvals_right = np.array([xvals[1] for xvals in self.recent_x_vals if xvals[1] is not None])
        best_xvals_left = np.average(recent_xvals_left, 
                                     weights=range(len(recent_xvals_right), 0, -1),
                                     axis=0)
        best_xvals_right = np.average(recent_xvals_right, 
                                      weights=range(len(recent_xvals_right), 0, -1),
                                      axis=0)
        if not np.all(np.isnan(best_xvals_left)) and not np.all(np.isnan(best_xvals_left)):
            self.best_x_vals = [best_xvals_left, best_xvals_right]
#         else:
#             ELSE USE PREVIOUS FITS
#             print(self.best_x_vals)
#             self.best_x_vals = [None, None]

        radii_curv_left = np.array([radii[0] for radii in self.radius_of_curvatures if radii[0] is not None])
        radii_curv_right = np.array([radii[1] for radii in self.radius_of_curvatures if radii[1] is not None])
        avg_radius_left = np.average(radii_curv_left,
                                     weights=range(len(radii_curv_left), 0, -1),
                                     axis=0)
        avg_radius_right = np.average(radii_curv_right,
                                      weights=range(len(radii_curv_right), 0, -1),
                                      axis=0)
        self.best_radius = 0.5*(avg_radius_left + avg_radius_right)
        
    
    def process_image(self, image):
        self.frame_count += 1
        
        ylen = image.shape[0]
        ploty = np.linspace(0, ylen-1, ylen)
        
        img_unwarp, left_fit, right_fit, left_fitx, right_fitx, img_filt = get_poly_lanes(image, prev_left_fit=self.best_fits[0],
                                                                                       prev_right_fit=self.best_fits[1])
        if left_fit is not None and right_fit is not None:
            left_curv_radius = calc_lane_curve_radius(left_fit, image.shape[0], 
                                                      self.xm_per_pix, self.ym_per_pix)
            right_curv_radius = calc_lane_curve_radius(right_fit, image.shape[0], 
                                                       self.xm_per_pix, self.ym_per_pix)

            curve_ratio = left_curv_radius/right_curv_radius
            lane_diff = (right_fitx - left_fitx) * self.xm_per_pix

            lane_width_base = lane_diff[-1] # lane width at base
            lane_width_max = np.max(lane_diff)
            lane_width_min = np.min(lane_diff)

            if max(left_curv_radius, right_curv_radius) > 1000:
                curve_ratio = 1.
        else:
            lane_width_base = lane_width_min = lane_width_max = curve_ratio = left_curv_radius = right_curv_radius = None
            
        if (left_fit is not None)  and (right_fit is not None) and (curve_ratio > 0.4) \
            and (curve_ratio < 2.5) and (lane_width_base > 3.0) and \
            (lane_width_base < 4.4) and (lane_width_min > 2.0):
            # fit looks good!
            self.recent_fits.append([left_fit, right_fit])
            self.recent_x_vals.append([left_fitx, right_fitx])
            self.radius_of_curvatures.append([left_curv_radius, right_curv_radius])
        else:
            # fit looks bad!
            self.recent_fits.append([None, None])
            self.recent_x_vals.append([None, None])
            self.radius_of_curvatures.append([None, None])
            
            ## PRINT STATS WHEN FIT DOESN'T PASS CHECKS
            print("Lane width base = ", lane_width_base)
            print("Lane width min = ", lane_width_min)
            print("Lane width max = ", lane_width_max)
            print("Curve Ratio = ", curve_ratio)
            
            ## SAVE IMAGES FOR DEBUGGING
#             img_wlanes_tmp = add_lanes_poly(image=image, left_fitx=left_fitx, right_fitx=right_fitx, y=ploty)            
#             plot_lane_lines(img_filt, left_fitx, right_fitx, ploty)            
#             cv2.putText(img_wlanes_tmp, "Radius of Curvature (Left) = {0:.1f}".format(left_curv_radius),
#                         bottomLeftCornerOfText, font, fontScale, fontColor, lineType)        
#             cv2.putText(img_wlanes_tmp, "Radius of Curvature (Right) = {0:.1f}".format(right_curv_radius),
#                         bottomLeftCornerOfText2, font, fontScale, fontColor, lineType)
#             cv2.putText(img_wlanes_tmp, "Base Lane Width, min, max = {0:.2f}".format(lane_width_base),
#                         bottomLeftCornerOfText3, font, fontScale, fontColor, lineType)
            
#             plt.imsave("debug/test_{}_grad.png".format(self.frame_count), img_filt)
#             plt.imsave("debug/test_{}_unwarp.png".format(self.frame_count), img_unwarp)
#             plt.imsave("debug/test_{}.png".format(self.frame_count), img_wlanes_tmp)
        
        self.update_best_fits()

        if self.best_x_vals[0] is not None:
                img_wlanes = add_lanes_poly(image=image, left_fitx=self.best_x_vals[0], right_fitx=self.best_x_vals[1], y=ploty)
        else:
            img_wlanes = add_lanes_poly(image=image, left_fitx=left_fitx, right_fitx=right_fitx, y=ploty)
        
        lane_center = 0.5*(self.best_x_vals[0][-1] + self.best_x_vals[1][-1])
        lane_offset = (image.shape[1]/2 - lane_center)*self.xm_per_pix
        
        cv2.putText(img_wlanes, "Radius of Curvature = {0:.1f}m".format(self.best_radius),
                    bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
        if lane_offset > 0:
            cv2.putText(img_wlanes, "Vehicle is {0:.1f}m right of center".format(abs(lane_offset)),
                        bottomLeftCornerOfText2, font, fontScale, fontColor, lineType)
        else:
            cv2.putText(img_wlanes, "Vehicle is {0:.2f}m left of center".format(abs(lane_offset)),
                        bottomLeftCornerOfText2, font, fontScale, fontColor, lineType)
        return img_wlanes

### Process Video

In [3]:
# inp_video = 'project_video.mp4'
inp_video = 'challenge_video.mp4'
# pkl_file = 'cameraCalibration_perspectiveTransform_matrices.pkl'
pkl_file = 'cameraCalibration_perspectiveTransform_matrices_challenge.pkl'

In [4]:
try:
    with open(pkl_file, 'rb') as f:
        data = pickle.load(f)
    mtx = data['mtx']
    dist = data['dist']
    M = data['M']
except FileNotFoundError:
    print("Run 'Advanced_LaneFinding - PreProcessing' to generate camera calibration, ",
          "perspective transform matrices before running this!")

In [5]:
# lane_detection = LaneFinding(xm_per_pix=3.7/700., ym_per_pix=36/720., n_hist=12)
lane_detection = LaneFinding(xm_per_pix=3.7/650., ym_per_pix=30/720., n_hist=25)

In [6]:
from moviepy.editor import VideoFileClip
clip1 = VideoFileClip(inp_video)#.subclip(0,5)
white_clip = clip1.fl_image(lane_detection.process_image)
%time white_clip.write_videofile("{}_out.mp4".format(inp_video.split('.')[0]), audio=False)
clip1.close()

Moviepy - Building video challenge_video_out.mp4.
Moviepy - Writing video challenge_video_out.mp4



t:  18%|████████████▏                                                       | 87/485 [00:11<00:53,  7.44it/s, now=None]

Lane width base =  3.27520572594
Lane width min =  3.26648149746
Lane width max =  5.53609807819
Curve Ratio =  0.350102781507


t:  18%|████████████▍                                                       | 89/485 [00:11<00:54,  7.27it/s, now=None]

Lane width base =  3.29767270112
Lane width min =  3.27654457474
Lane width max =  5.69568167953
Curve Ratio =  0.327688126511


...

t:  56%|█████████████████████████████████████▊                             | 274/485 [00:44<00:29,  7.05it/s, now=None]

Lane width base =  3.32594296412
Lane width min =  0.522686570308
Lane width max =  3.3537552028
Curve Ratio =  0.203647630152


t:  57%|█████████████████████████████████████▉                             | 275/485 [00:44<00:30,  6.85it/s, now=None]

Lane width base =  3.29389776947
Lane width min =  1.54544048714
Lane width max =  3.30711299537
Curve Ratio =  0.224230104777


Moviepy - Done !
Moviepy - video ready challenge_video_out.mp4
Wall time: 1min 14s


### Process Test Images

In [9]:
pkl_file = 'cameraCalibration_perspectiveTransform_matrices.pkl'

In [10]:
try:
    with open(pkl_file, 'rb') as f:
        data = pickle.load(f)
    mtx = data['mtx']
    dist = data['dist']
    M = data['M']
except FileNotFoundError:
    print("Run 'Advanced_LaneFinding - PreProcessing' to generate camera calibration, ",
          "perspective transform matrices before running this!")

In [11]:
test_images = [f for f in os.listdir("test_images") if f.endswith('.jpg') or f.endswith('.png')]
for test_image in test_images:
    lane_detection = LaneFinding()
    
    img = cv2.imread(os.path.join("test_images/", test_image))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    proc_img = lane_detection.process_image(img)
    proc_img = cv2.cvtColor(proc_img, cv2.COLOR_RGB2BGR)
    
    cv2.imwrite(os.path.join('output_images/', test_image), proc_img)